In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
%matplotlib inline

In [15]:
# There are two groups; control group and experimental group(here it is called patient)
# The response variable here is "Target" which indicates whether its corresponding patient_ID is in the biopsy image dataset
# 1 indicates that its patient_ID is in the biopsy image dataset
# Currently we don't have a full list of patient_IDs(from biopsy image dataset), as soon as we get the list, we can load it as 
# "biopsy_data" and we can perform RF to each group. 
# The below RF models are mockups. 

# Load and clean data 

In [2]:
#load data
csv_file = 'SEEM_bile_acid_data_patient.csv'
df_patient = pd.read_csv(csv_file, index_col=None)
csv_file2 = 'SEEM_bile_acid_data_control.csv'
df_control = pd.read_csv(csv_file2, index_col = None)
csv_file3 = 'temp.csv'
biopsy_data = pd.read_csv(csv_file3)

In [3]:
#remove 'Barcode','CMS ID','LC/MS code#
df_control = df_control.drop(['Barcode','CMS ID','LC/MS code#'], axis=1)
df_control = df_control.groupby('Patient ID',as_index=False).mean()
df_control['Target'] = 0
#
df_patient = df_patient.drop(['Barcode','CMS ID','LC/MS code#'], axis=1)
df_patient = df_patient.groupby('Patient ID',as_index=False).mean()
df_patient['Target'] = 0

# Populate "Reponse" value

In [4]:
index = []
def find_index(biopsy_data,biomaker_data):
    for i in range(0,len(biopsy_data)):
        for j in range(0,len(biomaker_data)):
            if biomaker_data['Patient ID'].loc[j] == biopsy_data['Patient ID'].loc[i]:
                index.append(j)

## Populdate "Response" value to patient set

In [5]:
find_index(biopsy_data,df_patient)
index = set(index)

In [6]:
for val in index:
    df_patient['Target'].loc[val] = 1

C:\Users\lukek\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## Populdate "Response" value to control set

In [7]:
index = []
find_index(biopsy_data,df_control)
index = set(index)

In [8]:
for val in index:
    df_control['Target'].loc[val] = 1

# Random Forest

## Random Forest to patient set

In [9]:
df_patient = df_patient.dropna()
X = df_patient.iloc[:,1:-1]
X = X.reset_index(drop=True)
y = df_patient.iloc[:,-1]
y = y.reset_index(drop=True)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)
## This line instantiates the model. 
rf = RandomForestClassifier() 
## Fit the model on your training data.
rf.fit(X_train, y_train) 
## And score it on your testing data.
rf.score(X_test, y_test)

0.9724770642201835

In [11]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

,importance
TCA,0.153448
Total CBA,0.120666
TCDCA,0.114820
Total UBA,0.110908
GCDCA,0.100414
Total BA,0.098588
CDCA,0.058888
GCA,0.054255
UBA/Total BA,0.043040
CBA/Total BA,0.041845


## Random Forest to patient set 

In [12]:
df_control = df_control.dropna()
X = df_control.iloc[:,1:-1]
X = X.reset_index(drop=True)
y = df_control.iloc[:,-1]
y = y.reset_index(drop=True)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)
## This line instantiates the model. 
rf = RandomForestClassifier() 
## Fit the model on your training data.
rf.fit(X_train, y_train) 
## And score it on your testing data.
rf.score(X_test, y_test)

1.0

In [14]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

,importance
TCA,0.0
UDCA,0.0
UBA/Total BA,0.0
CBA/UBA,0.0
Total UBA,0.0
Total CBA,0.0
Total BA,0.0
LCA,0.0
DCA,0.0
CDCA,0.0
